In [3]:
import os
from glob import glob
import numpy as np
import pandas as pd
from my_functions import *

In [22]:
path1 = '/Users/maggie/Documents/Github/HPD-Inference_and_Processing/Audio/Inference-Audio/Audio_CNN/H1-red/processed/'

x = 0
for numpy_file in glob(os.path.join(path1, '*.npy')):
    data = np.load(numpy_file)
    print(np.shape(data))


(492, 1000, 16)
(397, 1000, 16)
(458, 1000, 16)
(323, 1000, 16)
(496, 1000, 16)
(300, 1000, 16)
(303, 1000, 16)
(322, 1000, 16)
(323, 1000, 16)
(410, 1000, 16)


In [69]:
path2 = '/Volumes/TOSHIBA-21/H1-red/RS1/processed_audio/audio_downsampled/2019-11-06/'

for npz in glob(os.path.join(path2, '*.npz')):
    hour = os.path.basename(npz).split('_')[1]
    full_hr = np.load(npz)
#     print(type(audio_data.files))
    times = full_hr.files
    time_keys, audio_data = [], []
    for time in times:
        if len(full_hr[time]) > 0:
            time_keys.append(time)
            audio_data.append(full_hr[time])

    data = np.stack(audio_data)
    data = data.transpose(0,2,1)
    print(np.shape(data))

#         print(time, np.shape(audio_data[time]))
#     data = [audio_data[time] for time in times if len(audio_data[time]>0)]
#     print(np.shape(np.stack(data)))
    
    
#     for t in times:
#         print(type(t), t)
# #     print(time_keys)
    
#     
#     for time in times:
#         ds = np.asarray([audio_data[time]])
#         dst = ds.transpose(0,2,1)
        
        
        
#         keys = np.fromiter(Samples.keys(), dtype=float)
# vals = np.fromiter(Samples.values(), dtype=float)

(347, 16, 1000)
(350, 16, 1000)
(332, 16, 1000)
(329, 16, 1000)
(359, 16, 1000)
(329, 16, 1000)
(329, 16, 1000)
(354, 16, 1000)
(309, 16, 1000)
(344, 16, 1000)
(318, 16, 1000)
(328, 16, 1000)
(348, 16, 1000)
(359, 16, 1000)
(360, 16, 1000)
(347, 16, 1000)
(348, 16, 1000)
(358, 16, 1000)
(348, 16, 1000)
(351, 16, 1000)
(329, 16, 1000)
(359, 16, 1000)
(348, 16, 1000)
(345, 16, 1000)


In [ ]:
        hour = os.path.basename(npz).split('_')[1]
        audio_data = np.load(npz)
        times = audio_data.files

        preds = []

        for time in times:
            DCTs = audio_data[time]

In [ ]:
def open_npz(noise_file, path, audio_type='ds'):
    home = os.path.basename(path).split('-')[0]
    hub, day, time, action = get_title(noise_file).split(' ')

    file = f'{day}_{str(time[0:2])}00_{hub}_{home}_{audio_type}.npz'
    npz = os.path.join(path, hub, 'processed_audio', ps_type[audio_type], day, file)
    data = np.load(npz)  

In [2]:
import os
import sys
import time
import pickle
import numpy as np
import pandas as pd

from glob import glob
from datetime import datetime

from tensorflow.keras.models import model_from_json
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from my_functions import *

In [3]:
model_path = '/Users/maggie/Documents/Github/HPD-Inference_and_Processing/Audio/Inference-Audio/'
model = model_from_json(open(os.path.join(model_path, 'Audio_CNN/model-94_96/CNN_model.json')).read())
model.load_weights(os.path.join(model_path, 'Audio_CNN/model-94_96/CNN_weights.h5'))
model.summary()
scaler = pickle.load(open(os.path.join(model_path, 'Audio_CNN/model-94_96/scaler.pkl'), 'rb'))

Model: "CNN_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 997, 32)       544       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 996, 16)       2064      
_________________________________________________________________
flatten (Flatten)            (None, 191232)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 382466    
Total params: 385,074
Trainable params: 385,074
Non-trainable params: 0
_________________________________________________________________


In [7]:
# movement1 = '/Users/maggie/Desktop/plot_audio/filter_output/RS1-noise.npy'
# noSound = '/Users/maggie/Desktop/plot_audio/filter_output/RS1-quiet.npy'

f1 = '/Users/maggie/Documents/Github/HPD-Inference_and_Processing/Audio/Inference-Audio/Audio_CNN/CNN_testing_code/H1-red/processed/RS1-noise.npy'


def perform_inf(path):
    input_data = np.load(path) 
    input_data = np.transpose()
    ori_input_shape = input_data.shape
    input_data = input_data.reshape((len(input_data), -1))
    input_data = scaler.transform(input_data)
    input_data = input_data.reshape((len(input_data), ori_input_shape[1], ori_input_shape[2], 1))

    class_prob = model.predict(input_data)
    probabilities = class_prob[:,1]
    predictions = (class_prob>0.5).astype("int32")
    predictions = np.argmax(predictions, axis=1)
    print(probabilities, predictions)
    
perform_inf(f1)

ValueError: Error when checking input: expected conv2d_input to have shape (16, 1000, 1) but got array with shape (1000, 16, 1)